In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import shutil
import os
import random
import cv2
import numpy as np
import re
import copy
from typing import List, Dict, Any, Optional, Tuple
import math
import matplotlib.pyplot as plt

import numpy as np

import PIL
from PIL import Image, ImageChops, ImageEnhance, ImageFilter, ImageOps

In [2]:
%cd /content

/content


In [3]:
!git clone https://github.com/FahNos/svtr2_1_long_text.git

Cloning into 'svtr2_1_long_text'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 216 (delta 37), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (216/216), 1.02 MiB | 2.99 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [4]:
!pip install -r /content/svtr2_1_long_text/OpenOCR/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.2 MB/s eta 0:00:00


In [5]:
!pip install python-magic

# Download VietNamese data

In [6]:
%cd /content/svtr2_1_long_text/rec

/content/svtr2_1_long_text/rec


In [7]:
!python /content/svtr2_1_long_text/rec/data/download/download_dataset.py -c /content/svtr2_1_long_text/rec/configs/dataset/rec/ltb.yaml


[1/2] Downloading ../ltb/data.mdb from https://drive.usercontent.google.com/download?id=16AEA1YGTsyVB44uEjKi4ZUV1snjCYBr4&authuser=0&confirm=t ...
100% 74.4M/74.4M [00:01<00:00, 51.5MB/s]
Successfully downloaded ../ltb/data.mdb

[2/2] Downloading ../ltb/lock.mdb from https://drive.usercontent.google.com/download?id=1xU4OStrOaI23bPG4flWAPWn2YrQe2bmY&authuser=0&confirm=t ...
100% 8.19k/8.19k [00:00<00:00, 23.1MB/s]
Successfully downloaded ../ltb/lock.mdb
Downloads and extractions finished!


In [ ]:
!python /content/svtr2_1_long_text/rec/data/download/download_dataset.py -c /content/svtr2_1_long_text/rec/configs/dataset/rec/union14m_l_filtered.yaml


[1/18] Downloading ../Union14M-L-LMDB-Filtered/train_challenging/data.mdb from https://drive.usercontent.google.com/download?id=1etwzBgGHjsFsb0sygsaRnKbanW2PMe07&authuser=0&confirm=t ...
100% 1.56G/1.56G [00:22<00:00, 70.9MB/s]
Successfully downloaded ../Union14M-L-LMDB-Filtered/train_challenging/data.mdb

[2/18] Downloading ../Union14M-L-LMDB-Filtered/train_challenging/lock.mdb from https://drive.usercontent.google.com/download?id=1ly6FJfPjItwGlVQ-ifTrzzM3rVu3Ezhr&authuser=0&confirm=t ...
100% 8.19k/8.19k [00:00<00:00, 28.1MB/s]
Successfully downloaded ../Union14M-L-LMDB-Filtered/train_challenging/lock.mdb

[3/18] Downloading ../Union14M-L-LMDB-Filtered/train_challenging/rec_results.txt from https://drive.usercontent.google.com/download?id=10Xso9oYHgQf8BdYakD1JAZ8PP213tIQ6&authuser=0&confirm=t ...
100% 18.2M/18.2M [00:00<00:00, 61.7MB/s]
Successfully downloaded ../Union14M-L-LMDB-Filtered/train_challenging/rec_results.txt

[4/18] Downloading ../Union14M-L-LMDB-Filtered/train_easy/dat

In [8]:
!python /content/svtr2_1_long_text/rec/data/download/download_dataset.py -c /content/svtr2_1_long_text/rec/configs/dataset/rec/evaluation.yaml


[1/12] Downloading ../evaluation/IC15_1811/data.mdb from https://drive.usercontent.google.com/download?id=1eGY0kXNV1qVxeUpoGzs-ioUO-ky7msH6&authuser=0&confirm=t ...
100% 21.9M/21.9M [00:01<00:00, 16.8MB/s]
Successfully downloaded ../evaluation/IC15_1811/data.mdb

[2/12] Downloading ../evaluation/IC15_1811/lock.mdb from https://drive.usercontent.google.com/download?id=1BWv7aLoLAT7avY326gXP3GJF48UZpuBC&authuser=0&confirm=t ...
100% 8.19k/8.19k [00:00<00:00, 30.8MB/s]
Successfully downloaded ../evaluation/IC15_1811/lock.mdb

[3/12] Downloading ../evaluation/SVT/data.mdb from https://drive.usercontent.google.com/download?id=1ecEZ4cJ7dIbTCZRltE0s5KzUotQWagH-&authuser=0&confirm=t ...
100% 6.79M/6.79M [00:00<00:00, 33.2MB/s]
Successfully downloaded ../evaluation/SVT/data.mdb

[4/12] Downloading ../evaluation/SVT/lock.mdb from https://drive.usercontent.google.com/download?id=1OygBP7i9R-3Pwi6WodCcW31J8CUMugOJ&authuser=0&confirm=t ...
100% 8.19k/8.19k [00:00<00:00, 24.6MB/s]
Successfully downlo

In [ ]:
# @title Create lmdb VNese dataset
# import json

# def read_crop_label(filepaths, output_filename):
#     """
#     Đọc dữ liệu từ các file, tạo file JSON với định dạng:
#     {"filename": "path/to/image", "text": "label"}
#     """
#     n_lines = 0
#     samples = 0
#     try:
#         with open(output_filename, 'w', encoding='utf-8') as f_out:
#             for filepath in filepaths:
#                 try:
#                     with open(filepath, 'r', encoding='utf-8') as f_in:
#                         for line in f_in:
#                             n_lines += 1
#                             line = line.strip()
#                             if line:
#                                 parts = line.split('\t')
#                                 if len(parts) == 2:
#                                     image_path, label = parts
#                                     if len(label) > 200 or len(label) == 0:
#                                         # print(f"Dòng len(label) > 200: {line}, len = {len(label)}")
#                                         continue
#                                     json_data = {"filename": image_path, "text": label}
#                                     json.dump(json_data, f_out, ensure_ascii=False)
#                                     f_out.write('\n')
#                                     samples += 1
#                                 # else:
#                                 #     print(f"Dòng không hợp lệ: {line}, len = {len(parts)}")
#                 except FileNotFoundError:
#                     print(f"Không tìm thấy file: {filepath}")
#                 except Exception as e:
#                     print(f"Lỗi khi đọc file {filepath}: {e}")
#         print(f"Đã tạo file JSON: {output_filename}")
#     except Exception as e:
#         print(f"Lỗi tổng quan: {e}")
#     return n_lines, samples

# filepath = ["/content/Raw_vnese_data/data/crop_label.txt",
#             "/content/Raw_vnese_data/VietOCR-Paddle/InkData_line_processed/rec/rec_gt_train_ink.txt",
#             "/content/Raw_vnese_data/VietOCR-Paddle/en_00/rec/rec_gt_train_en_00.txt",
#             "/content/Raw_vnese_data/VietOCR-Paddle/en_01/rec/rec_gt_train_en01.txt",
#             "/content/Raw_vnese_data/VietOCR-Paddle/meta/rec/rec_gt_train_meta.txt",
#             "/content/Raw_vnese_data/VietOCR-Paddle/random/rec/rec_gt_train_random.txt",
#             "/content/Raw_vnese_data/VietOCR-Paddle/vi_00/rec/rec_gt_train_vi00.txt",
#             "/content/Raw_vnese_data/VietOCR-Paddle/vi_01/rec/rec_gt_train_vi01.txt"]

# output_filename = "/content/vnese.json"
# n_lines, samples = read_crop_label(filepath, output_filename)
# print(f"Tổng số dòng: {n_lines}")     # Tổng số dòng: 702152
# print(f"Tổng số mẫu hợp lệ samples: {samples}")     # Tổng số mẫu hợp lệ samples: 701870

In [ ]:
# @title Create lmdb VNese dataset

# %cd rec

# !python /content/rec/data/create_lmdb_dataset.py \
# -l /content/vnese.json \
# -s /content/Union14M-L-LMDB-Filtered/train_vnese \
# -m 200

# Load github repository

# Training

In [9]:
!pip install "numpy<2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 107.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
%cd /content/svtr2_1_long_text/OpenOCR

!python -m torch.distributed.launch\
 --nproc_per_node=1 tools/train_rec.py\
 --c /content/svtr2_1_long_text/OpenOCR/configs/rec/svtrv2.1/focal_svtrv2_smtr_gtc_rctc_small.yml

# Inference

In [10]:
!mkdir -p /content/svtr2_1_long_text/OpenOCR/pretrained_model/

In [24]:
%cd /content/svtr2_1_long_text/OpenOCR

!python tools/eval_rec.py --c /content/svtr2_1_long_text/OpenOCR/configs/rec/svtrv2.1/focal_svtrv2_smtr_gtc_rctc_small_eval.yml

/content/svtr2_1_long_text/OpenOCR
[2025/05/07 08:28:19] openrec INFO: ----------- Config -----------
[2025/05/07 08:28:19] openrec INFO: Architecture : 
[2025/05/07 08:28:19] openrec INFO:     Decoder : 
[2025/05/07 08:28:19] openrec INFO:         ctc_decoder : 
[2025/05/07 08:28:19] openrec INFO:             name : RCTCDecoder
[2025/05/07 08:28:19] openrec INFO:         detach : False
[2025/05/07 08:28:19] openrec INFO:         gtc_decoder : 
[2025/05/07 08:28:19] openrec INFO:             ds : True
[2025/05/07 08:28:19] openrec INFO:             max_len : 200
[2025/05/07 08:28:19] openrec INFO:             name : SMTRDecoder
[2025/05/07 08:28:19] openrec INFO:             next_mode : True
[2025/05/07 08:28:19] openrec INFO:             num_layer : 1
[2025/05/07 08:28:19] openrec INFO:             sub_str_len : 5
[2025/05/07 08:28:19] openrec INFO:         infer_gtc : True
[2025/05/07 08:28:19] openrec INFO:         name : GTCDecoder
[2025/05/07 08:28:19] openrec INFO:     Encoder : 